In [34]:
from keras.layers import Input, Dense
from keras.models import Model

In [35]:
from keras.datasets import mnist
import numpy as np

In [36]:
(x_train, _), (x_test, _) = mnist.load_data()

In [37]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [38]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)

In [39]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.3605 - val_loss: 0.2652
Epoch 2/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2579 - val_loss: 0.2517
Epoch 3/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2452 - val_loss: 0.2393
Epoch 4/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.2351 - val_loss: 0.2278
Epoch 5/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.2223 - val_loss: 0.2118
Epoch 6/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.2027 - val_loss: 0.1958
Epoch 7/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1874 - val_loss: 0.1812
Epoch 8/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1787 - val_loss: 0.1745
Epoch 9/100
60000/60000 [==============================] - 4s 69us/step - loss: 0.1729 - val_loss: 0.1

60000/60000 [==============================] - 4s 69us/step - loss: 0.1043 - val_loss: 0.1028
Epoch 78/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1041 - val_loss: 0.1020
Epoch 79/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1039 - val_loss: 0.1032
Epoch 80/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1036 - val_loss: 0.1027
Epoch 81/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1034 - val_loss: 0.1042
Epoch 82/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1031 - val_loss: 0.1018
Epoch 83/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1030 - val_loss: 0.1025
Epoch 84/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1029 - val_loss: 0.1037
Epoch 85/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1024 - val_loss: 0.1026
Epoch 86/100
60000/60000 [========================

In [25]:
autoencoder